In [ ]:
# (EXAMING VQGAN FIRST STEP MODEL)

import torch

# device = torch.cuda.device(0)
device = torch.device('cuda:0')
# eval testing

# load config for transformer
from omegaconf import OmegaConf

# ----------------------------------------------------------
config_path = "configs/owt_vqgan.yaml"
# ----------------------------------------------------------
# ----------------------------------------------------------
ckpt_path = f"logs/allusc_vqgan_1024_baseline/checkpoints/last.ckpt"
# ----------------------------------------------------------

config = OmegaConf.load(config_path)
import yaml
import torch.nn.functional as F
# print(yaml.dump(OmegaConf.to_container(config)))
config ['data']['params']['batch_size'] = 1

# ----------------------------------------------------------
# config['data']['params']['train']['params']['dataroot'] = "/home/ICT2000/chenh/Haiwei/Datasets/OWT/USC_Galen_Center"
# ----------------------------------------------------------

unconditional = config.model.params.cond_stage_config == "__is_unconditional__"

from taming.models.vqgan import VQModel
model = VQModel(**config.model.params).to(device)

# loading checkpoint
import torch
sd = torch.load(ckpt_path, map_location=device)["state_dict"]
missing, unexpected = model.load_state_dict(sd, strict=False)
seg_model = False

In [ ]:
# dataset instantiation
import importlib
from torch.utils.data import random_split, DataLoader, Dataset

def instantiate_from_config(config):
    if not "target" in config:
        raise KeyError("Expected key `target` to instantiate.")
    return get_obj_from_str(config["target"])(**config.get("params", dict()))


def get_obj_from_str(string, reload=False):
    module, cls = string.rsplit(".", 1)
    if reload:
        module_imp = importlib.import_module(module)
        importlib.reload(module_imp)
    return getattr(importlib.import_module(module, package=None), cls)

data = instantiate_from_config(config.data)
data.prepare_data()
data.setup()
dataset = data.datasets['train']
dataset_iter = iter(data._train_dataloader())


In [ ]:
from PIL import Image
import numpy as np
import cv2

import torch.nn.functional as F

idx = 14
batch = dataset[idx]
print(batch.keys())

tensify = lambda x: torch.from_numpy(x[None]).to(device).permute(0,3,1,2).contiguous().float()
tensor_to_numpy = lambda x:x.detach().cpu().numpy()[0].transpose(1,2,0)

def write_images(path, image, n_row=1):
    image = ((image + 1) * 255 / 2).astype(np.uint8)
    if image.ndim == 3:
        if image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if image.shape[2] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGRA)
    cv2.imwrite('{}'.format(str(path)), np.squeeze(image))

def show_image(s, iftorch=False):
  if iftorch:
    s = s.detach().cpu().numpy()[0]
  s = (1 + np.clip(s, -1, 1)) / 2 # (s - s.min()) / (s.max() - s.min())
  s = (s * 255).astype(np.uint8)
  s = Image.fromarray(s)
  display(s)
    
# segmentation = batch['segmentation']
# show_image(segmentation)

image = batch['image']
show_image(image)
# write_images('test.png',image)
# write_images('test_seg.png',segmentation)


In [ ]:
# test: tokenization with regard to inpainting


def CentralMask(s, offset=None):
    mask = np.ones((s, s), np.uint8)
    if offset is None:
        offset = s // 4
    else:
        offset = max(min(offset, 0), s//2 - 1)
    mask[offset:offset+s//2, offset:offset+s//2] = 0
    return mask[..., np.newaxis].astype(np.float32)

# first tokenize the original input
x = tensify(image) 
quant_z, _, info = model.encode(x)
_, _, indices = info

# then masked out central area of the image and tokenize it again
x_masked = tensify(image * CentralMask(image.shape[0]))
quant_z_masked, _, info_masked = model.encode(x_masked)
_, _, indices_masked = info_masked


In [ ]:
# reconstruct with the tokens
img_recon = model.decode(quant_z)
show_image(tensor_to_numpy(img_recon), False)

# reconstructed masked image
img_mask_recon = model.decode(quant_z_masked)
show_image(tensor_to_numpy(img_mask_recon) * CentralMask(image.shape[0]), False)

# masked image
show_image(image * CentralMask(image.shape[0]))


In [ ]:
import numpy as np

def print_index_grid(grid, horizontal_bar=False):
    for row in grid:
        row_str = ""
        for number in row:
            row_str += '{0:<5}|'.format(number)
        if horizontal_bar:
            hori = ""
            for i in range(len(row_str)):
                hori += '_'
            print(hori)
        print(row_str)

indices_masked_np = indices_masked.reshape(16,16).detach().cpu().numpy()
indices_np = indices.reshape(16,16).detach().cpu().numpy()

print_index_grid(indices_np)
print("---------------------------------------------------------------------------------------")
print_index_grid(indices_masked_np)

state_grid = list()

sim_grid = indices_np == indices_masked_np
print("---------------------------------------------------------------------------------------")
print_index_grid(sim_grid)


In [ ]:

# initialize some params
mask = 1.0
mask_tensor = None
nb = 1 #c_code.shape[0]
codebook_size = config.model.params.first_stage_config.params.embed_dim
# z_indices_shape = c_indices.shape
c_code_res = 16
res = 256
z_indices_shape = [nb,res]
z_code_shape = [nb, codebook_size,c_code_res,c_code_res]
print(z_code_shape, z_indices_shape)


In [ ]:
# (caution! slow execution) (optional) batch image of the original size
image_path = dataset[idx]['img_path']
full_image = Image.open(image_path)
full_img = np.array(full_image).astype(np.float32) 
full_img = (img/127.5 - 1.0)
show_image(full_img)

In [ ]:
# (optional) sample random indices to image VQQGAN
# z_code_shape = codebook_size
z_indices = torch.randint(codebook_size, z_indices_shape, device=model.device)
print(z_indices.shape)
x_sample = model.decode_to_img(z_indices, z_code_shape)
print(x_sample.shape)
if seg_model:
    x_sample = F.softmax(x_sample,dim=1)
show_image(tensor_to_numpy(x_sample), False)

In [ ]:
# (optional) sample random indices to image VQQGAN
@torch.no_grad()
def decode_to_img(model, index, zshape, use_softmax=False):
    bhwc = (zshape[0],zshape[2],zshape[3],zshape[1])
    quant_z = model.quantize.get_codebook_entry(
        index.reshape(-1), shape=bhwc)
    x = model.decode(quant_z)
    return x

codebook_size = config.model.params.embed_dim
c_code_res = 8
res = 128
z_indices_shape = [nb,c_code_res**2]
z_code_shape = [nb, codebook_size,c_code_res,c_code_res]
z_indices = torch.randint(codebook_size, z_indices_shape, device=model.device)
x_sample = decode_to_img(model, z_indices, z_code_shape)
print(z_indices.shape)
show_image(tensor_to_numpy(x_sample), False)

#(optional VQGAN testing)
x = tensify(batch['image']) 
quant_z, _, info = model.encode(x)
indices = info[2].view(quant_z.shape[0], -1)
x_sample = decode_to_img(model, indices, z_code_shape)
show_image(tensor_to_numpy(x_sample), False)

In [ ]:
# USE THIS IF NO SEGMENTATION

from PIL import Image
import numpy as np
import cv2

import torch.nn.functional as F

idx = 5
batch = dataset[idx]
print(batch.keys())

tensify = lambda x: torch.from_numpy(x[None]).to(device).permute(0,3,1,2).contiguous().float()
tensor_to_numpy = lambda x:x.detach().cpu().numpy()[0].transpose(1,2,0)

def write_images(path, image, n_row=1):
    image = ((image + 1) * 255 / 2).astype(np.uint8)
    if image.ndim == 3:
        if image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if image.shape[2] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGRA)
    cv2.imwrite('{}'.format(str(path)), np.squeeze(image))

def show_image(s, iftorch=False):
  if iftorch:
    s = s.detach().cpu().numpy()[0]
  s = ((s + 1) * 255 / 2).astype(np.uint8)
  s = Image.fromarray(s)
  display(s)
    
image = batch['image']
show_image(image)

# initialize some params
mask = 1.0
mask_tensor = None
nb = 1 #c_code.shape[0]


In [ ]:
# decoder structure testing
device = feat.device
weight = torch.rand(256, 128, 2, 2).float()
mapping_conv = torch.nn.Conv2d(256, 128, kernel_size=5, stride=1, padding=2)
feat = quant_z # nb, nc, h, w
mapping_conv = mapping_conv.to(device)
print(feat.shape)
mapping_conv(feat).shape

In [ ]:
res = 16
for i, i_level in enumerate(reversed(range(4))):
    print(i_level, i)
    res *= 2
print(res)

In [ ]:
class RestrictedUpconv2D(torch.nn.Module):
    def __init__(self, c_in, c_out, stride=2, activation='relu'):
        super().__init__()
        if isinstance(stride, tuple):
            assert len(stride) == 2
            sx, sy = stride
        else:
            assert isinstance(stride, int)
            sx = sy = stride
        self.mlp = torch.nn.Conv2d(c_in, c_out*sx*sy, kernel_size=1, bias=False)
        self.sx = sx
        self.sy = sy

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.mlp(x) # B, C*sx*sy, H, W 
        x = x.reshape(B, -1, self.sx, self.sy, H, W)
        x = x.permute(0,1,4,2,5,3).reshape(B, -1, self.sx*H, self.sy*W)
        return x
    
feat_ones = torch.zeros_like(feat).to(device)
feat_ones[0,0,1,4] = 1.0
feat_ones[0,0,1,3] = 1.0
feat_ones[0,0,3,5] = 1.0

upconv = torch.nn.Sequential(LocalizedUpconv2D(256, 128).to(device),
                             LocalizedUpconv2D(128, 64).to(device))
upfeat = upconv(feat_ones)

In [ ]:
# visualization
featmap = upfeat[0,0].unsqueeze(-1).detach().cpu().numpy()
featmap = (featmap - featmap.min()) / (featmap.max() - featmap.min())
import matplotlib.pyplot as plt
plt.imshow(featmap)

In [ ]:
# conditional model or segmentation model
if not unconditional:
    # reconstructing seg tensor
    seg_tensor = tensify(segmentation)
    c_code, c_indices = model.encode_to_c(seg_tensor)
    seg_rec = model.cond_stage_model.decode(c_code)
    seg_rec = F.softmax(seg_rec,dim=1)
    show_image(tensor_to_numpy(seg_rec), False)
    
if seg_model:
    # reconstructing seg tensor
    seg_tensor = tensify(segmentation)
    c_code, c_indices = model.encode_to_z(seg_tensor)
    seg_rec = model.first_stage_model.decode(c_code)
    seg_rec = F.softmax(seg_rec,dim=1)
    show_image(tensor_to_numpy(seg_rec), False)

In [ ]:
# inpainting testing: replace random z indices with that of an encoded batch image
key = 'segmentation' if seg_model else "image"
x = tensify(batch[key]) 
if unconditional:
    c = x
    quant_c, c_indices = model.encode_to_c(c)
else:
    c = tensify(batch['segmentation'])
    quant_c = c_code
# c = x if unconditional else tensify(batch['segmentation'])
quant_z, z_indices = model.encode_to_z(x)
# quant_c, c_indices = model.encode_to_c(c)
gt_z_indices = z_indices

In [ ]:
# (optional) mlp testing

import kornia
import torch.distributions as dists

def flip_positions(positions):
    positions = positions.transpose(2,3)
    positions = torch.cat([positions[:,0][:,None],-positions[:,1][:,None]],1)
    return positions.contiguous()


def get_position(size, dim, device, batch_size):
    height, width = size
    aspect_ratio = width / height
    position = kornia.utils.create_meshgrid(width, height, device=torch.device(device)).permute(0, 3, 1, 2)
    position[:, 1] = -position[:, 1] * aspect_ratio  # flip y axis

    if dim == 1:
        x, y = torch.split(position, 1, dim=1)
        position = x
    if dim == 3:
        x, y = torch.split(position, 1, dim=1)
        z = torch.ones_like(x) * torch.rand(1, device=device) * 2 - 1
        a = torch.randint(0, 3, (1,)).item()
        if a == 0:
            xyz = [x, y, z]
        elif a == 1:
            xyz = [z, x, y]
        else:
            xyz = [x, z, y]
        # xyz =  [x,z,y]
        position = torch.cat(xyz, dim=1)
    position = position.expand(batch_size, dim, width, height)
    return position if dim == 3 else flip_positions(position)

def get_distribution_type(shape, type='uniform'):
    if type == 'normal':
        return dists.Normal(torch.zeros(shape), torch.ones(shape))
    elif type == 'uniform':
        return dists.Uniform(torch.zeros(shape) - 1, torch.ones(shape))
    else:
        raise NotImplementedError

def get_cropped_coord_grid(device, 
                           dist_shift, 
                           nb,  
                           res=256, 
                           crop_res=128, 
                           scale=2.0):

    coord_grid = scale * get_position([res, res], 2, device, nb)
    # dist_shift = get_distribution_type([1,2], type='uniform')
    coord_grid_sample = torch.zeros([nb, 2, crop_res, crop_res]).to(device)
    shift = 0.5 * (dist_shift.sample([nb]) + 1) * (res - crop_res)
    shift = shift.int().reshape(nb,2)
    for idx, s in enumerate(shift):
        dx,dy = s
        dx = int(dx.item())
        dy = int(dy.item())
        coord_grid_sample[idx] = coord_grid[idx, :, dx:dx+crop_res, dy:dy+crop_res]
    return coord_grid_sample, shift

def crop_input(device, x, shift, nb, crop_res=128):
    new_x = torch.zeros([nb, 3, crop_res, crop_res]).to(device)
    for idx, s in enumerate(shift):
        dx, dy = s
        dx = int(dx.item())
        dy = int(dy.item())       
        new_x[idx] = x[idx,:,dx:dx+crop_res,dy:dy+crop_res]
    return new_x

def batched_index_select_2d(index, feature):
    batch_size, image_dim, res1, res2 = index.shape
    _, f_dim, grid_dim, _ = feature.shape
    indexf = index.reshape(batch_size,image_dim,-1)
    indexf2 = indexf[:,0] * grid_dim + indexf[:,1]
    def select(x, idx):
        x_select = torch.index_select(x.reshape(f_dim, -1),1,idx)
        x_select = x_select.reshape(f_dim, res1, res2)
        return x_select[None]
    xyz_select = torch.cat([select(x,idx) for x,idx in zip(feature, indexf2)], dim=0)
    return xyz_select

def stationary_noise(positions, feature, scale=2.0, sigma=0.2, mode='gaussian'):
    '''
    positions: nb,2,w,h
    codebook feature: nb,nc,c_w,c_h (assume to be defining the feature space in [-scale, scale])
    '''
    # cgs_q: coordinate_grid_sampled_quantized (according to codebook resolution)
    c_res = feature.shape[2]
    cgs_q = (positions + scale) * c_res / (2 * scale)
    
    # index-select from features    
    idx_1 = torch.clamp(torch.floor(cgs_q).long(), 0, c_res - 1)
    idx_2 = torch.clamp(idx_1 + 1, 0, c_res - 1)
    idx_3 = torch.clamp(torch.cat([idx_1[:,0].unsqueeze(1) + 1, idx_1[:,1].unsqueeze(1)],1), 0, c_res-1)
    idx_4 = torch.clamp(torch.cat([idx_1[:,0].unsqueeze(1), idx_1[:,1].unsqueeze(1)+1],1), 0, c_res-1)

    # distance to corners
    px = cgs_q[:,0]
    py = cgs_q[:,1]
    x1 = (px - torch.floor(px))
    x2 = (torch.floor(px) + 1 - px)
    y1 = (py - torch.floor(py))
    y2 = (torch.floor(py) + 1 - py)  
    
    if mode == 'gaussian':
        f_grouped = torch.cat([batched_index_select_2d(index, feature)[...,None] \
                    for index in [idx_1,idx_2,idx_3,idx_4]],dim=-1)
        dist1 = torch.sqrt(x1**2 + y1**2)[...,None]
        dist2 = torch.sqrt(x2**2 + y2**2)[...,None]
        dist3 = torch.sqrt(x2**2 + y1**2)[...,None]
        dist4 = torch.sqrt(x1**2 + y2**2)[...,None]
        dists = torch.cat([dist1,dist2,dist3,dist4], 3)
        dists = torch.nn.functional.softmax(-dists/sigma, -1).unsqueeze(1)
        return torch.sum(dists * f_grouped, -1)
    elif mode == 'bilinear' or mode == 'linear':
        tr,tl,br,bl = [batched_index_select_2d(index, feature)\
                    for index in [index_1,index_3,index_4,index_2]]
        bx = x1.unsqueeze(1) * bl + x2.unsqueeze(1) * br
        tx = x1.unsqueeze(1) * tl + x2.unsqueeze(1) * tr        
        return y1.unsqueeze(1) * bx + y2.unsqueeze(1) * tx        
    else:
        raise NotImplementedError(f"type of interpolation {mode} is not recognized!")

def disabled_train(self, mode=True):
    """Overwrite model.train with this function to make sure train/eval mode
    does not change anymore."""
    return self

# x -> [cos(2^0 pi x), ..., sin(2^9) pi x]
# b,dim,... -> b,2*dim*l,...
def positional_encoding(x, l=5, beta=None):
    bs,dim = x.shape[:2]
    res = x.shape[2:]
    x = x.unsqueeze(2).expand(bs,dim,l,*res)
    octaves = 2**(torch.arange(l)).to(x.device)
    if beta is not None:        
        octaves = octaves * beta
    for r in res:
        octaves = octaves.unsqueeze(-1)
    x = x * octaves[None,None,...] * np.pi
    x = torch.cat((torch.sin(x).unsqueeze(2), torch.cos(x).unsqueeze(2)),2)
    return x.reshape(bs,-1,*res)


In [ ]:
quant_z.shape, x.shape

In [ ]:
# visualizing features
from matplotlib import pyplot as plt

def tensor_to_visual(img, normalize=False):
    signed_to_unsigned = lambda x : (x + 1) / 2
    img = img[0].detach().cpu().numpy()
    if img.min() < 0:
        img = signed_to_unsigned(img)
    # img = (img - img.min()) / (img.max() - img.min())    
    if normalize:
        img = (img - img.min()) / (img.max() - img.min() + 1e-4)
    img = np.clip(img,0.0,1.0)
    return img.transpose(1,2,0)

z_original = F.interpolate(quant_z, [256, 256])
z_img = tensor_to_visual(z_original[:,:3], True)
plt.imshow(z_img, interpolation='nearest')
plt.show()

In [ ]:
dist_shift = get_distribution_type([1,2], type='uniform')
scale = 2.0
# # random cropping to model stationary shift
cgs, shift = get_cropped_coord_grid(quant_z.device, 
                                    dist_shift, 
                                    quant_z.shape[0], 
                                    x.shape[2], 
                                    128, 
                                    scale)
cropped_x = crop_input(quant_z.device, x, shift, quant_z.shape[0], crop_res=128)
cgs = scale * get_position([256, 256], 2, x.device, x.shape[0])
#feature sampling
feat = stationary_noise(cgs, quant_z, scale=scale)  
feat_vis = tensor_to_visual(feat[:,:3], True)
plt.imshow(feat_vis, interpolation='nearest')
plt.show()

In [ ]:
## initialize some other params related to transformer
from IPython.display import clear_output
import time
import os

output_dir = os.path.join("logs", "eval", "notebook_vis")
os.makedirs(output_dir, exist_ok=True)

idx = z_indices
idx = idx.reshape(z_code_shape[0],z_code_shape[2],z_code_shape[3])

if not unconditional:
    cidx = c_indices
    cidx = cidx.reshape(c_code.shape[0],c_code.shape[2],c_code.shape[3])

temperature = 1.0
top_k = 100
update_every = 10

nx = z_code_shape[2]
ny = z_code_shape[3]

In [ ]:
# inpainting testing: replace random z indices with that of an encoded batch image

MASK_LABEL = 1


key = 'segmentation' if seg_model else "image"
x = tensify(batch[key]) 
if unconditional:
    c = x
    quant_c, c_indices = model.encode_to_c(c)
else:
    c = tensify(batch['segmentation'])
    quant_c = c_code
# c = x if unconditional else tensify(batch['segmentation'])
quant_z, z_indices = model.encode_to_z(x)
# quant_c, c_indices = model.encode_to_c(c)
gt_z_indices = z_indices

# inpainting demo: masking out a certain region of the image
seg_label = np.argmax(segmentation, axis=2)
mask = (seg_label != MASK_LABEL).astype(np.float32)[...,None]
show_image(segmentation * mask)
show_image(image * mask)

# idx, cidx: codebook of the encoded image
mask_tensor = torch.from_numpy(mask).to(idx.device).float()
mask_tensor = mask_tensor.unsqueeze(0)[...,0]
mask_tensor = torch.nn.functional.interpolate(mask_tensor.unsqueeze(1), scale_factor=1/16)
mask_tensor = mask_tensor[:,0].int()
# print(mask_tensor.reshape(-1))
# print(mask_tensor.shape, idx.shape)

# mask idx 
idx = idx * mask_tensor

In [ ]:
# generate a multiplied-scale scene with a large (partially fixed) random codebook
multiplier = 1
nnx = nx * multiplier
nny = ny * multiplier
c_code_res = 16
step_size = 8
no_mask = False
det = False

target_image = np.zeros([multiplier*256, multiplier*256, 3])
z_indices = torch.randint(codebook_size, [nb,(multiplier**2)*res], device=model.device)
z_indices = z_indices.reshape(nb, nnx, nny)

# (demo) masking gt_z_indices
if mask_tensor is not None:
    input_z_indices = gt_z_indices * mask_tensor.reshape(1, -1)
else:
    input_z_indices = gt_z_indices
    
# partially fill the z_indices with true data
z_indices[:,:nx, :ny] = input_z_indices.reshape(nb, nx, ny)
occupancy = np.zeros(z_indices.shape).astype(bool)

if no_mask:
    pass
elif mask_tensor is not None:
    occupancy[:,:nx, :ny] = mask_tensor.bool().detach().cpu().numpy()
else:
    occupancy[:,:nx, :ny] = True

start_t = time.time()

# outer loop: image to decode
for i in range(0, nnx - step_size, step_size):
    for j in range(0, nny - step_size, step_size):
        idx = z_indices[:, i:i+c_code_res, j:j+c_code_res].reshape(nb, -1)
        occ = occupancy[:, i:i+c_code_res, j:j+c_code_res].reshape(nb, -1)       
        # only update a block in idx if it is not occupied
        for ii in range(idx.shape[1]):
            if not occ[0,ii]:
                if unconditional:
                    patch = torch.cat((c_indices, idx[:,:ii]), dim=1)
                    logits,_ = model.transformer(patch)
                else:
                    cpatch = cidx[:, i:i+c_code_res, j:j+c_code_res].reshape(nb, -1)
                    # cpatch = cidx.reshape(nb, -1)
                    logits,_ = model.transformer(idx[:,:ii], cpatch)
                # patch = torch.cat((c_indices, idx[:,:ii]),1)
                # logits,_ = model.transformer(patch)
                logits = logits[:, -1, :]
                logits = logits/temperature
                if top_k is not None:
                  logits = model.top_k_logits(logits, top_k)
                probs = torch.nn.functional.softmax(logits, dim=-1)
                if det:
                    _, idx[:,ii] = torch.topk(probs, k=1, dim=-1)
                else:
                    idx[:,ii] = torch.multinomial(probs, num_samples=1)
                occ[:,ii] = True
        idx = idx.reshape(nb, c_code_res, c_code_res)
        occ = occ.reshape(nb, c_code_res, c_code_res)
        z_indices[:, i:i+c_code_res, j:j+c_code_res] = idx
        occupancy[:, i:i+c_code_res, j:j+c_code_res] = occ
        print(f"Time: {time.time() - start_t} seconds")
        print(f"Step: ({i},{j})")

new_z_code_shape = [nb, codebook_size, multiplier*16, multiplier*16]
x_sample = model.decode_to_img(z_indices, new_z_code_shape)
if seg_model:
    x_sample = F.softmax(x_sample, dim=1)
target_image = x_sample[0].detach().cpu().numpy().transpose(1,2,0)
show_image(segmentation)
show_image(image * mask)
show_image(target_image, False)

In [ ]:
for i in range(0, nnx, c_code_res):
    for j in range(0, nny, c_code_res):
        patch_code = z_indices[:, i:i+c_code_res, j:j+c_code_res]
        x_sample = model.decode_to_img(patch_code, z_code_shape)
        if seg_model:
            x_sample = F.softmax(x_sample, dim=1)
        patch_image = x_sample[0].detach().cpu().numpy().transpose(1,2,0)
        target_image[i*16:i*16+256, j*16:j*16+256] = patch_image
show_image(target_image, False)

In [ ]:
# generate the original scene with a known encoded codebook
update_every = 1000000000
multiplier = 1
quant_z, z_indices = model.encode_to_z(x)
idx = z_indices
idx = idx.reshape(z_code_shape[0], z_code_shape[2],z_code_shape[3])

nx = z_code_shape[2]
ny = z_code_shape[3]
start_t = time.time()

for i in range(0, nx-0):
  if i <= 8:
    local_i = i
  elif nx-i < 8:
    local_i = 16-(nx-i)
  else:
    local_i = 8
  for j in range(0,ny-0):
    if j <= 8:
      local_j = j
    elif ny-j < 8:
      local_j = 16-(ny-j)
    else:
      local_j = 8
    
    i_start = i-local_i
    i_end = i_start+16
    j_start = j-local_j
    j_end = j_start+16
    # print(i_start, i_end, j_start, j_end)
    patch = idx[:,i_start:i_end,j_start:j_end]
    patch = patch.reshape(patch.shape[0],-1)
    if unconditional:
        patch = torch.cat((c_indices, patch), dim=1)
        logits,_ = model.transformer(patch[:,:-1])
    else:
        cpatch = cidx[:, i_start:i_end, j_start:j_end]
        cpatch = cpatch.reshape(cpatch.shape[0], -1)
        # patch = torch.cat((cpatch, patch), dim=1)
        logits,_ = model.transformer(patch[:,:-1], cpatch)

    logits = logits[:, -256:, :]
    logits = logits.reshape(z_code_shape[0],16,16,-1)
    logits = logits[:,local_i,local_j,:]
    logits = logits/temperature
    
    if top_k is not None:
      logits = model.top_k_logits(logits, top_k)

    probs = torch.nn.functional.softmax(logits, dim=-1)
    idx[:,i,j] = torch.multinomial(probs, num_samples=1)

    step = i*z_code_shape[3]+j
    if step%update_every==0 or step==z_code_shape[2]*z_code_shape[3]-1:
      x_sample = model.decode_to_img(idx, z_code_shape)
      clear_output()
      print(f"Time: {time.time() - start_t} seconds")
      print(f"Step: ({i},{j}) | Local: ({local_i},{local_j}) | Crop: ({i_start}:{i_end},{j_start}:{j_end})")
      print(x_sample.shape)
      if not unconditional:
          show_image(tensor_to_numpy(seg_rec), False)
      show_image(tensor_to_numpy(x_sample), False)
      # write_images(os.path.join(output_dir, f"step{step}_{i}_{j}.png"), tensor_to_numpy(x_sample))
      patch_output = tensor_to_numpy(x_sample)
